In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [9]:
tf.keras.backend.clear_session()

In [10]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [11]:
subject = 'Mato Grosso - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [12]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_MT.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Consumo de Cimento (t)
0,2008,6.535602e+07,2.793553e+06,17.127903,5.260918e+07,0.752766,0.787367,0.732454,5.483335e+08,2.546644e+09,...,0.738916,8.063005,31.686523,993.754162,1421.360784,26.585520,19.247754,68.306560,1.265376,994.957000
1,2009,6.593967e+07,2.815133e+06,17.131657,5.267158e+07,0.753704,0.788247,0.735581,5.302764e+08,2.679945e+09,...,0.738104,8.095019,26.992644,992.577520,971.624734,26.514656,18.393406,65.147890,1.139212,998.465000
2,2010,6.789156e+07,2.861375e+06,17.294037,5.339572e+07,0.754460,0.788564,0.738478,5.046425e+08,2.891930e+09,...,0.737478,8.057848,25.116066,989.658472,896.219335,26.516966,17.684947,62.460338,1.378783,1099.340000
3,2011,6.985811e+07,2.887266e+06,17.471861,5.415240e+07,0.755039,0.788328,0.741142,4.854622e+08,3.091988e+09,...,0.737032,8.047069,28.216671,987.048009,916.320612,26.594241,18.048644,63.561988,1.548197,1204.074000
4,2012,7.159016e+07,2.905726e+06,17.591515,5.472408e+07,0.755027,0.787468,0.742721,4.867661e+08,3.189506e+09,...,0.736572,6.695389,27.946087,986.033104,884.118586,26.504045,18.419653,65.231377,1.625213,1381.539000
5,2013,7.314074e+07,2.912179e+06,17.684470,5.519726e+07,0.756113,0.786174,0.746782,5.225848e+08,3.093872e+09,...,0.736953,7.068881,31.557800,985.745432,817.482501,26.560721,18.794569,66.303790,1.668395,1618.657000
6,2014,7.450695e+07,2.898181e+06,17.765377,5.559725e+07,0.755956,0.784438,0.748013,5.676413e+08,2.779340e+09,...,0.736866,7.495401,33.403862,985.582958,728.555092,26.792583,18.998037,66.249639,1.689375,1411.234429
7,2015,7.552129e+07,2.867943e+06,17.804826,5.583983e+07,0.755452,0.782141,0.749014,5.179392e+08,2.561731e+09,...,0.736688,7.838256,31.029765,985.297803,631.112819,26.741398,18.889658,66.109773,1.670487,1241.100343
8,2016,7.631332e+07,2.837856e+06,17.844276,5.605555e+07,0.758777,0.788475,0.756866,4.524144e+08,2.516859e+09,...,0.732916,7.730867,27.875013,985.139748,546.211949,26.647262,18.703122,65.821838,1.617712,1083.128229
9,2017,7.659073e+07,2.826355e+06,17.841399,5.610212e+07,0.757848,0.785609,0.756036,4.213689e+08,2.497710e+09,...,0.733636,7.251143,26.025417,985.051490,572.018116,26.728560,18.583504,65.279143,1.559939,1220.058000


In [13]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,6.535602e+07,2.793553e+06,17.127903,5.260918e+07,0.752766,0.787367,0.732454,5.483335e+08,2.546644e+09,1.874883e+10,...,869.834453,0.738916,8.063005,31.686523,993.754162,1421.360784,26.585520,19.247754,68.306560,1.265376
1,6.593967e+07,2.815133e+06,17.131657,5.267158e+07,0.753704,0.788247,0.735581,5.302764e+08,2.679945e+09,2.055015e+10,...,949.083048,0.738104,8.095019,26.992644,992.577520,971.624734,26.514656,18.393406,65.147890,1.139212
2,6.789156e+07,2.861375e+06,17.294037,5.339572e+07,0.754460,0.788564,0.738478,5.046425e+08,2.891930e+09,2.342247e+10,...,1038.662223,0.737478,8.057848,25.116066,989.658472,896.219335,26.516966,17.684947,62.460338,1.378783
3,6.985811e+07,2.887266e+06,17.471861,5.415240e+07,0.755039,0.788328,0.741142,4.854622e+08,3.091988e+09,2.604265e+10,...,1134.526356,0.737032,8.047069,28.216671,987.048009,916.320612,26.594241,18.048644,63.561988,1.548197
4,7.159016e+07,2.905726e+06,17.591515,5.472408e+07,0.755027,0.787468,0.742721,4.867661e+08,3.189506e+09,2.708753e+10,...,1291.161206,0.736572,6.695389,27.946087,986.033104,884.118586,26.504045,18.419653,65.231377,1.625213
5,7.314074e+07,2.912179e+06,17.684470,5.519726e+07,0.756113,0.786174,0.746782,5.225848e+08,3.093872e+09,2.523394e+10,...,1371.926397,0.736953,7.068881,31.557800,985.745432,817.482501,26.560721,18.794569,66.303790,1.668395
6,7.450695e+07,2.898181e+06,17.765377,5.559725e+07,0.755956,0.784438,0.748013,5.676413e+08,2.779340e+09,1.954361e+10,...,1268.814714,0.736866,7.495401,33.403862,985.582958,728.555092,26.792583,18.998037,66.249639,1.689375
7,7.552129e+07,2.867943e+06,17.804826,5.583983e+07,0.755452,0.782141,0.749014,5.179392e+08,2.561731e+09,1.327596e+10,...,1123.345627,0.736688,7.838256,31.029765,985.297803,631.112819,26.741398,18.889658,66.109773,1.670487
8,7.631332e+07,2.837856e+06,17.844276,5.605555e+07,0.758777,0.788475,0.756866,4.524144e+08,2.516859e+09,1.011370e+10,...,1075.219903,0.732916,7.730867,27.875013,985.139748,546.211949,26.647262,18.703122,65.821838,1.617712
9,7.659073e+07,2.826355e+06,17.841399,5.610212e+07,0.757848,0.785609,0.756036,4.213689e+08,2.497710e+09,8.456309e+09,...,1111.839984,0.733636,7.251143,26.025417,985.051490,572.018116,26.728560,18.583504,65.279143,1.559939


In [14]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      998.465000
1     1099.340000
2     1204.074000
3     1381.539000
4     1618.657000
5     1411.234429
6     1241.100343
7     1083.128229
8     1220.058000
9     1316.386000
10    1446.064000
11    1555.266000
12    1788.146000
13    1947.017000
14            NaN
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [15]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.740588,-1.627429,-1.768928,-1.746450,-1.736363,0.561629,-1.727009,1.204752,-0.990234,0.166091,...,-1.925963,1.450893,1.102745,1.214956,2.196775,2.448959,-0.498590,1.635424,1.610412,-1.533856
1,-1.598275,-1.055001,-1.755010,-1.698578,-1.152572,0.837117,-1.343029,0.820657,-0.546688,0.428911,...,-1.344018,1.051022,1.170358,-0.661837,1.783361,0.503680,-0.709406,-0.183740,0.176596,-2.299018
2,-1.122342,0.171630,-1.152996,-1.143071,-0.682504,0.936364,-0.987463,0.275392,0.158672,0.847995,...,-0.686213,0.743048,1.091854,-1.412165,0.757751,0.177523,-0.702533,-1.692261,-1.043365,-0.846063
3,-0.642835,0.858410,-0.493731,-0.562596,-0.322949,0.862562,-0.660386,-0.132596,0.824346,1.230291,...,0.017744,0.523822,1.069089,-0.172424,-0.159437,0.264469,-0.472647,-0.917841,-0.543293,0.181401
4,-0.220508,1.348072,-0.050124,-0.124039,-0.329912,0.593304,-0.466463,-0.104860,1.148829,1.382743,...,1.167958,0.297449,-1.785600,-0.280614,-0.516025,0.125183,-0.740974,-0.127851,0.214493,0.648489
5,0.157572,1.519248,0.294500,0.238952,0.345134,0.188062,0.031981,0.657046,0.830615,1.112297,...,1.761039,0.484935,-0.996801,1.163487,-0.617099,-0.163043,-0.572364,0.670458,0.701294,0.910382
6,0.490699,1.147933,0.594458,0.545796,0.247355,-0.355312,0.183203,1.615453,-0.215960,0.282052,...,1.003861,0.442014,-0.096011,1.901614,-0.674184,-0.547688,0.117409,1.103701,0.676713,1.037619
7,0.738027,0.345838,0.740713,0.731890,-0.065842,-1.074422,0.306105,0.558230,-0.940033,-0.632425,...,-0.064359,0.354512,0.628086,0.952359,-0.774374,-0.969163,-0.034864,0.872930,0.613224,0.923072
8,0.931149,-0.452241,0.886969,0.897373,2.001700,0.908478,1.270060,-0.835563,-1.089341,-1.093812,...,-0.417760,-1.501484,0.401284,-0.309032,-0.829906,-1.336391,-0.314910,0.475739,0.482521,0.603000
9,0.998790,-0.757316,0.876303,0.933102,1.423819,0.011185,1.168125,-1.495939,-1.153056,-1.335633,...,-0.148848,-1.147363,-0.611873,-1.048571,-0.860916,-1.224770,-0.073054,0.221037,0.236176,0.252619


In [16]:
reshaped_train_input = dfToInputRNN(train_input)

In [17]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.91769583, -0.66096769,  0.83348718,  0.88011305,
         -0.30046596, -1.46823039,  1.00934571, -1.0453709 ,
          1.21859064, -0.99976018,  0.79428371,  1.15575749,
         -0.52559858,  1.30260559, -0.55253046, -0.25130359,
          1.38787176,  1.47154451, -1.80649616,  1.09296291,
          0.94281138,  0.52836084,  0.61258103,  0.87656257,
          1.47801536,  1.71982086,  1.28221181,  1.39086472,
          1.58209844,  1.08872091,  0.55212962, -1.81398118,
          1.44113674,  1.11191094,  1.30470835,  0.86390516,
         -1.67365968, -1.70738939, -1.55813196, -0.48498301,
         -0.75714639,  0.40770893, -0.27394734,  2.02323219,
          1.2908951 ,  1.70465513,  1.81148084,  1.39968653,
         -1.36191442,  1.28024593,  1.11054795,  0.12662799,
          1.40414112,  1.40468999,  1.45643959,  1.58509336,
          1.32283157,  1.30222047,  1.55138602,  1.91801602,
         -0.77489784,  2.32098884, -1.1942541 , -1.17081042,
          0.46480286, -1

In [18]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0      998.465000
1     1099.340000
2     1204.074000
3     1381.539000
4     1618.657000
5     1411.234429
6     1241.100343
7     1083.128229
8     1220.058000
9     1316.386000
10    1446.064000
11    1555.266000
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [19]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    1555.266
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [20]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.765494,-0.551552,0.800035,0.794933,-1.569273,-2.551374,0.831359,1.108405,2.635823,-0.189959,...,1.207226,-1.55722,-1.265069,-0.730185,0.611707,1.620843,2.316005,-1.810579,-2.274914,0.168633


In [21]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.76549398, -0.55155174,  0.80003549,  0.79493323,
         -1.56927324, -2.55137369,  0.83135931,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
          1.20722572, -1

In [22]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    1788.146
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [36]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [37]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [38]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[321951205, 1275697826, 248621864, 108742067, 1421009054, 3152240367, 3066398808, 4110065189, 3256566884, 2233888390, 373460523, 2087337784, 2903552278, 809065269, 1515000104, 3288583525, 3278540623, 3982690551, 122074427, 2255836218, 3357790222, 2726718506, 3248272124, 1557739931, 2018988900, 3270632900, 3856736139, 871695833, 175293240, 2704458192, 3419481473, 392649903, 4286809905, 1079558985, 3944165675, 262029506, 1058183513, 1330704217, 3104841170, 938630687, 3383109182, 2342826802, 4050107229, 3066540627, 3419954402, 1479364519, 3426522736, 868731599, 433098055, 3305790138, 2896581605, 2135954617, 2803725728, 890393349, 2531819475, 793392096, 1391226050, 2027343226, 1963550163, 1314818976, 1284972990, 2832706825, 2976011043, 3885228137, 1077253678, 1863414865, 1920294280, 302383953, 2067254714, 2452794109, 1428960057, 4029863921, 2815466139, 2023265606, 4098470673, 805836756, 1593221325, 3830090764, 3652847360, 2662079464, 2899471326, 1389363929, 237807250, 893288782, 1676356671

loss: 821.055908203125


Step: 89 ___________________________________________
loss: 1177.7135009765625


Step: 90 ___________________________________________
loss: 806.2317504882812


Step: 91 ___________________________________________
loss: 1337.3614501953125


Step: 92 ___________________________________________
loss: 782.796875


Step: 93 ___________________________________________
loss: 445.4801330566406


Step: 94 ___________________________________________
loss: 1117.1138916015625


Step: 95 ___________________________________________
loss: 836.3544921875


Step: 96 ___________________________________________
loss: 889.787841796875


Step: 97 ___________________________________________
loss: 1291.3018798828125


Step: 98 ___________________________________________
loss: 888.1133422851562


Step: 99 ___________________________________________
loss: 662.8579711914062


final_seed: 4029863921


In [39]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 1718365.5000
Epoch 2/10000
1/1 [==============================] - 0s 5ms/step - loss: 1679806.3750
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 1494506.6250
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 365453.0938
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 72758.4609
Epoch 6/10000
1/1 [==============================] - 0s 5ms/step - loss: 53690.2070
Epoch 7/10000
1/1 [==============================] - 0s 5ms/step - loss: 50375.4492
Epoch 8/10000
1/1 [==============================] - 0s 5ms/step - loss: 35116.3398
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 28880.1309
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 37833.8242
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 37400.9961
Epoch 12/10000
1/1 [==============================] - 0s 5ms/step - 

1/1 [==============================] - 0s 4ms/step - loss: 1613.3629
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 9467.1211
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 3974.7939
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 3248.7703
Epoch 102/10000
1/1 [==============================] - 0s 5ms/step - loss: 7031.8628
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 6698.3022
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 4529.0747
Epoch 105/10000
1/1 [==============================] - 0s 5ms/step - loss: 3030.0090
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 5649.0176
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 6810.0020
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - loss: 6608.7505
Epoch 109/10000
1/1 [==============================] - 0s 4ms/step - loss: 6831.24

1/1 [==============================] - 0s 5ms/step - loss: 5346.5747
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 7412.8359
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 4668.6011
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 2847.3496
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 4991.4897
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 6011.4707
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 5893.6470
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 6114.0015
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 1597.5812
Epoch 203/10000
1/1 [==============================] - 0s 5ms/step - loss: 6337.7915
Epoch 204/10000
1/1 [==============================] - 0s 4ms/step - loss: 5137.3291
Epoch 205/10000
1/1 [==============================] - 0s 4ms/step - loss: 6736.6

1/1 [==============================] - 0s 4ms/step - loss: 3870.6248
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 6239.3892
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 2322.4768
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 5998.9746
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 5806.3848
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 6208.4590
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 3294.0400
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 5542.6089
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 3098.2366
Epoch 299/10000
1/1 [==============================] - 0s 4ms/step - loss: 2996.2605
Epoch 300/10000
1/1 [==============================] - 0s 4ms/step - loss: 5944.1538
Epoch 301/10000
1/1 [==============================] - 0s 4ms/step - loss: 2060.9

1/1 [==============================] - 0s 4ms/step - loss: 3912.4343
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 6113.2593
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 2774.8572
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 4975.5933
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 7694.6470
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 11456.5820
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 3485.7917
Epoch 393/10000
1/1 [==============================] - 0s 4ms/step - loss: 2825.4539
Epoch 394/10000
1/1 [==============================] - 0s 4ms/step - loss: 4243.4863
Epoch 395/10000
1/1 [==============================] - 0s 4ms/step - loss: 3683.2512
Epoch 396/10000
1/1 [==============================] - 0s 4ms/step - loss: 4662.9780
Epoch 397/10000
1/1 [==============================] - 0s 4ms/step - loss: 4464.

...

1/1 [==============================] - 0s 4ms/step - loss: 4306.6777
Epoch 1438/10000
1/1 [==============================] - 0s 5ms/step - loss: 7530.9702
Epoch 1439/10000
1/1 [==============================] - 0s 4ms/step - loss: 8126.2720
Epoch 1440/10000
1/1 [==============================] - 0s 4ms/step - loss: 4440.0220
Epoch 1441/10000
1/1 [==============================] - 0s 4ms/step - loss: 3814.6003
Epoch 1442/10000
1/1 [==============================] - 0s 5ms/step - loss: 5761.6753
Epoch 1443/10000
1/1 [==============================] - 0s 4ms/step - loss: 6628.2739
Epoch 1444/10000
1/1 [==============================] - 0s 4ms/step - loss: 3339.8250
Epoch 1445/10000
1/1 [==============================] - 0s 5ms/step - loss: 3318.1101
Epoch 1446/10000
1/1 [==============================] - 0s 4ms/step - loss: 4412.3003
Epoch 1447/10000
1/1 [==============================] - 0s 4ms/step - loss: 2478.1912
Epoch 1448/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 1876.1670
Epoch 1533/10000
1/1 [==============================] - 0s 4ms/step - loss: 1675.4927
Epoch 1534/10000
1/1 [==============================] - 0s 4ms/step - loss: 2866.9153
Epoch 1535/10000
1/1 [==============================] - 0s 4ms/step - loss: 2195.5183
Epoch 1536/10000
1/1 [==============================] - 0s 4ms/step - loss: 5064.6045
Epoch 1537/10000
1/1 [==============================] - 0s 4ms/step - loss: 4200.7109
Epoch 1538/10000
1/1 [==============================] - 0s 4ms/step - loss: 3042.2722
Epoch 1539/10000
1/1 [==============================] - 0s 4ms/step - loss: 6227.8442
Epoch 1540/10000
1/1 [==============================] - 0s 4ms/step - loss: 3276.2129
Epoch 1541/10000
1/1 [==============================] - 0s 5ms/step - loss: 4350.2026
Epoch 1542/10000
1/1 [==============================] - 0s 4ms/step - loss: 1327.4296
Epoch 1543/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 6302.6255
Epoch 1628/10000
1/1 [==============================] - 0s 4ms/step - loss: 1866.7350
Epoch 1629/10000
1/1 [==============================] - 0s 4ms/step - loss: 6580.2241
Epoch 1630/10000
1/1 [==============================] - 0s 4ms/step - loss: 3220.5449
Epoch 1631/10000
1/1 [==============================] - 0s 4ms/step - loss: 1544.2643
Epoch 1632/10000
1/1 [==============================] - 0s 4ms/step - loss: 5403.1787
Epoch 1633/10000
1/1 [==============================] - 0s 4ms/step - loss: 4508.0200
Epoch 1634/10000
1/1 [==============================] - 0s 4ms/step - loss: 2825.0183
Epoch 1635/10000
1/1 [==============================] - 0s 4ms/step - loss: 2296.8826
Epoch 1636/10000
1/1 [==============================] - 0s 4ms/step - loss: 5946.2749
Epoch 1637/10000
1/1 [==============================] - 0s 4ms/step - loss: 6669.4531
Epoch 1638/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 6146.5854
Epoch 1723/10000
1/1 [==============================] - 0s 4ms/step - loss: 2630.3665
Epoch 1724/10000
1/1 [==============================] - 0s 4ms/step - loss: 2953.1777
Epoch 1725/10000
1/1 [==============================] - 0s 5ms/step - loss: 3217.7617
Epoch 1726/10000
1/1 [==============================] - 0s 4ms/step - loss: 2804.0527
Epoch 1727/10000
1/1 [==============================] - 0s 4ms/step - loss: 4092.4707
Epoch 1728/10000
1/1 [==============================] - 0s 4ms/step - loss: 5891.1362
Epoch 1729/10000
1/1 [==============================] - 0s 4ms/step - loss: 5019.6519
Epoch 1730/10000
1/1 [==============================] - 0s 4ms/step - loss: 2221.7834
Epoch 1731/10000
1/1 [==============================] - 0s 4ms/step - loss: 6761.7036
Epoch 1732/10000
1/1 [==============================] - 0s 4ms/step - loss: 6353.6582
Epoch 1733/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 5039.8730
Epoch 1818/10000
1/1 [==============================] - 0s 4ms/step - loss: 3890.3945
Epoch 1819/10000
1/1 [==============================] - 0s 4ms/step - loss: 4697.5542
Epoch 1820/10000
1/1 [==============================] - 0s 4ms/step - loss: 4079.8311
Epoch 1821/10000
1/1 [==============================] - 0s 4ms/step - loss: 2460.4719
Epoch 1822/10000
1/1 [==============================] - 0s 4ms/step - loss: 3233.6492
Epoch 1823/10000
1/1 [==============================] - 0s 4ms/step - loss: 4720.5469
Epoch 1824/10000
1/1 [==============================] - 0s 5ms/step - loss: 2035.2490
Epoch 1825/10000
1/1 [==============================] - 0s 5ms/step - loss: 4889.4238
Epoch 1826/10000
1/1 [==============================] - 0s 4ms/step - loss: 2406.7078
Epoch 1827/10000
1/1 [==============================] - 0s 13ms/step - loss: 4303.7148
Epoch 1827: early stopping


In [40]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 619ms/step
[1788.146](test_target) - [1526.8514](prediction) = 261.29455957031246


In [41]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.14612596486545978

In [42]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [43]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.958981,-1.053327,-0.731205,-0.792708,-1.266344,-1.367241,-1.240125,1.148205,-1.122043,-1.121418,...,-1.199016,1.274172,-0.544470,1.358778,1.020728,1.401690,1.413631,1.261025,1.256285,0.043458
1,-0.420657,-0.290573,-0.682732,-0.617899,0.087939,0.370587,0.031362,0.140893,-0.184474,-0.185492,...,-0.049930,-0.105707,1.402573,-0.339841,0.337331,-0.538217,-0.741954,-0.076111,-0.065726,-1.245895
2,1.379638,1.343901,1.413937,1.410608,1.178406,0.996653,1.208763,-1.289098,1.306518,1.306910,...,1.248946,-1.168466,-0.858104,-1.018937,-1.358059,-0.863473,-0.671678,-1.184915,-1.190558,1.202438


train_target:


,Mato Grosso - Consumo de Cimento (t)
0,998.465
1,1099.340
2,1204.074


test_input:


...

,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.765494,-0.551552,0.800035,0.794933,-1.569273,-2.551374,0.831359,1.108405,2.635823,-0.189959,...,1.207226,-1.55722,-1.265069,-0.730185,0.611707,1.620843,2.316005,-1.810579,-2.274914,0.168633


test_target:


,Mato Grosso - Consumo de Cimento (t)
12,1788.146


1/1 [==============================] - 1s 575ms/step
Error: 261.29455957031246




[1174.87939453125,
 1348.0556640625,
 1431.27099609375,
 1395.2327880859375,
 1224.4793701171875,
 1080.0091552734375,
 1181.120361328125,
 1236.77294921875,
 1414.575927734375,
 1526.8514404296875]

In [44]:
display(targets)
display(predictions)

[1381.539,
 1618.657,
 1411.2344285714282,
 1241.1003428571426,
 1083.1282285714285,
 1220.058,
 1316.386,
 1446.064,
 1555.266,
 1788.146]

[1174.87939453125,
 1348.0556640625,
 1431.27099609375,
 1395.2327880859375,
 1224.4793701171875,
 1080.0091552734375,
 1181.120361328125,
 1236.77294921875,
 1414.575927734375,
 1526.8514404296875]

In [45]:
mae = mean_absolute_error(predictions, targets)
mae

167.93712617187506

In [46]:
porcentage = mae/np.mean(targets)
porcentage

0.11942977824316535